In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.datasets import fetch_openml
import pickle
from scipy.special import expit
from sklearn.model_selection import train_test_split

## Open MNIST data and create a subset with desired digits

<font color=red>Change the code below to select a different subset </font>

In [2]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
y = np.asarray([int(numeric_string) for numeric_string in y])

# Saving the temporary variables for fast retrieval
with open('temp.pickle', 'wb') as handle:
    pickle.dump([X, y], handle)


In [3]:
# loading the temporary variables for fast retrieval
with open('temp.pickle', 'rb') as handle:
    X, y = pickle.load(handle)

X = X
    
Nclasses = 3
labelclasses = y<Nclasses
Xnew = X[labelclasses]
ynew = y[labelclasses]
Nfeatures = np.size(Xnew,1)
Nsamples = np.size(Xnew,0)


In [4]:
 
X_train, X_test, y_train, y_test = train_test_split(Xnew, ynew, test_size=0.3, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.14, random_state=1) # 0.25 x 0.8 = 0.2

Nsamples = X_train.shape[0]
Nfeatures = X_train.shape[1]

X_train = X_train.T
X_val = X_val.T
X_test = X_test.T
